# Exercise - SVM

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. This is a binary classification task: predict whether a loan will be bad or not (1=Yes, 0=No). This is an important task for banks to prevent bad loans from being issued.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

## Goal

Use the **loan.csv** data set and build a model to predict **BAD**. 

Since you have a relatively small data set, I recommend using cross-validation to evaluate your accuracy.

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [2]:
#We will predict the "price" value in the data set:

loan = pd.read_csv("loan.csv")
loan.head()

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,0,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,1,50000,220528.0,300900.0,HomeImp,Self,5.0,0.0,0.0,0.000000,0.0,2.0,NaN
3,1,22400,51470.0,68139.0,DebtCon,Mgr,9.0,0.0,0.0,31.168696,2.0,8.0,37.952180
4,0,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076


# Split data (train/test)

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(loan, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [5]:
train_target = train['BAD']
test_target = test['BAD']

train_inputs = train.drop(['BAD'], axis=1)
test_inputs = test.drop(['BAD'], axis=1)

##  Identify the numeric, binary, and categorical columns

In [6]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [7]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [8]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [9]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [10]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [11]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [12]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

array([[-0.31412013, -1.30301181, -0.86148829, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.45733454,  0.7398414 ,  0.58636192, ...,  1.        ,
         0.        ,  0.        ],
       [-1.10330939,  0.2001631 ,  0.18146318, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.21657988, -0.83000156, -0.82081329, ...,  0.        ,
         0.        ,  0.        ],
       [-0.46486414,  1.79196675,  1.36974799, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31412013, -0.08740643, -0.21782887, ...,  0.        ,
         0.        ,  0.        ]])

In [13]:
train_x.shape

(1750, 20)

# Tranform: transform() for TEST

In [14]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

array([[ 0.06717356,  0.36706438,  0.32127798, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.32432512,  0.57631513,  0.42769944, ...,  0.        ,
         1.        ,  0.        ],
       [-0.33185472,  0.41209537,  0.12227549, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.8993616 , -0.51071616, -0.32326299, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.82706576, -0.8366813 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5446698 , -0.06422056, -0.11380525, ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
test_x.shape

(750, 20)

# Calculate the Baseline

In [16]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_target)

DummyClassifier(strategy='most_frequent')

In [17]:
from sklearn.metrics import accuracy_score

In [18]:
dummy_train_pred = dummy_clf.predict(train_x)
baseline_train_acc = accuracy_score(train_target, dummy_train_pred)
print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

# Baseline is 0: 59.82% (not BAD)

Baseline Train Accuracy: 0.6034285714285714


In [19]:
dummy_test_pred = dummy_clf.predict(test_x)
baseline_test_acc = accuracy_score(test_target, dummy_test_pred)
print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

# Baseline is 0: 58.93% (not BAD)

Baseline Test Accuracy: 0.5773333333333334


# Train an SVM model with linear kernel

In [20]:
from sklearn.svm import SVC
lin_svm = SVC(kernel="linear")
lin_svm.fit(train_x, train_target)

SVC(kernel='linear')

### Calculate the accuracy

In [21]:
from sklearn.metrics import accuracy_score

In [22]:
train_y_pred = lin_svm.predict(train_x)
#Train accuracy
accuracy_score(train_target, train_y_pred)

0.7525714285714286

In [23]:
test_y_pred = lin_svm.predict(test_x)
#Test accuracy
accuracy_score(test_target, test_y_pred)

# Train is slightly > test -> no overfitting
# Better than baseline

0.7266666666666667

# Train an SVM model with poly kernel

In [24]:
from sklearn.svm import SVC
pol_svm = SVC(kernel="poly", degree=3, coef0=1, C=0.1)
pol_svm.fit(train_x, train_target)

SVC(C=0.1, coef0=1, kernel='poly')

### Calculate the accuracy

In [25]:
train_y_pred = pol_svm.predict(train_x)
#Train accuracy
accuracy_score(train_target, train_y_pred)

0.7925714285714286

In [26]:
test_y_pred = pol_svm.predict(test_x)
#Test accuracy
accuracy_score(test_target, test_y_pred)

# With degree=3 and C of 10 -> major overfitting
# Less overfitting with C of 0.1

0.7493333333333333

# Train an SVM model with rbf kernel

In [27]:
rbf_svm = SVC(kernel="rbf", C=.1, gamma='scale')

rbf_svm.fit(train_x, train_target)

SVC(C=0.1)

### Calculate the accuracy

In [28]:
train_y_pred = rbf_svm.predict(train_x)
#Train accuracy
accuracy_score(train_target, train_y_pred)

0.7634285714285715

In [29]:
test_y_pred = rbf_svm.predict(test_x)
#Test accuracy
accuracy_score(test_target, test_y_pred)

# Overfitting with C=10 and gamma = scale
# Better with C=0.1

0.724

# Optional: try grid search on one of your SVM models

In [30]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import random

param_distribs = {
        'C': randint(low=5, high=50),
        'gamma': uniform(0.1, 0.5),    
    }

rbf_svm_grid = SVC(kernel="rbf", decision_function_shape='ovr')

rbf_search = RandomizedSearchCV(rbf_svm_grid, param_distributions=param_distribs,
                                n_iter=20, cv=5, scoring='accuracy', random_state=42)

rbf_search.fit(train_x, train_target)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000020D6D65BC40>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020D6802DCF0>},
                   random_state=42, scoring='accuracy')

In [31]:
cvres = rbf_search.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

0.8651428571428571 {'C': 43, 'gamma': 0.49827149343011645}
0.8634285714285713 {'C': 19, 'gamma': 0.4659969709057026}
0.8497142857142859 {'C': 25, 'gamma': 0.17800932022121826}
0.8411428571428571 {'C': 23, 'gamma': 0.14998745790900145}
0.8668571428571429 {'C': 15, 'gamma': 0.5330880728874676}
0.8462857142857143 {'C': 40, 'gamma': 0.1714334089609704}
0.82 {'C': 7, 'gamma': 0.11029224714790123}
0.8617142857142858 {'C': 6, 'gamma': 0.4609993861334124}
0.8491428571428571 {'C': 34, 'gamma': 0.20616955533913808}
0.8617142857142858 {'C': 25, 'gamma': 0.4087407548138583}
0.8297142857142857 {'C': 26, 'gamma': 0.10353315260985871}
0.8542857142857144 {'C': 29, 'gamma': 0.24561457009902096}
0.8377142857142857 {'C': 46, 'gamma': 0.12333283160680772}
0.8565714285714285 {'C': 20, 'gamma': 0.21638567021515212}
0.8617142857142858 {'C': 48, 'gamma': 0.35711721920680584}
0.868 {'C': 7, 'gamma': 0.5299702033681603}
0.8445714285714286 {'C': 11, 'gamma': 0.18526206184364577}
0.8302857142857144 {'C': 43, 'gam

In [32]:
final_model = rbf_search.best_estimator_

test_predictions = final_model.predict(test_x)
#Test accuracy
accuracy_score(test_target, test_predictions)

0.8893333333333333